In [1]:
import csv

In [2]:
import numpy as np
import os
import shutil

In [3]:
import pandas as pd
import xarray as xr
from netCDF4 import Dataset
import netCDF4

In [16]:
dir_path="/gpfs/work/sharmas/sim_data_1/data432"
file_list=sorted(os.listdir(dir_path))

In [17]:
file_list

['boxwd_lwc2000_rm10_nu00_xi1024',
 'boxwd_lwc2000_rm10_nu30_xi1024',
 'boxwd_lwc2000_rm10_nu40_xi1024',
 'boxwd_lwc2000_rm15_nu00_xi1024',
 'boxwd_lwc2000_rm15_nu40_xi1024']

In [10]:
file_list

['boxwd_lwc2000_rm10_nu00_xi1024',
 'boxwd_lwc2000_rm10_nu40_xi1024',
 'boxwd_lwc2000_rm15_nu00_xi1024',
 'boxwd_lwc2000_rm15_nu40_xi1024']

In [15]:
for filename in file_list:
    num=len(os.listdir(dir_path+filename))
    if num>4:
        print(f"File Name:{filename} | Number of files:{num}")
        #shutil.rmtree(dir_path+filename)

File Name:data432 | Number of files:5
File Name:data433 | Number of files:5
File Name:data434 | Number of files:5
File Name:data435 | Number of files:5
File Name:data436 | Number of files:5
File Name:data437 | Number of files:5
File Name:data438 | Number of files:5
File Name:data439 | Number of files:5
File Name:data440 | Number of files:5
File Name:data441 | Number of files:5
File Name:data442 | Number of files:5
File Name:data443 | Number of files:5
File Name:data444 | Number of files:5
File Name:data445 | Number of files:5
File Name:data446 | Number of files:5
File Name:data447 | Number of files:5
File Name:data448 | Number of files:5
File Name:data449 | Number of files:5
File Name:data450 | Number of files:5
File Name:data451 | Number of files:5
File Name:data452 | Number of files:5
File Name:data453 | Number of files:5
File Name:data454 | Number of files:5
File Name:data455 | Number of files:5
File Name:data456 | Number of files:5
File Name:data457 | Number of files:5
File Name:da

In [18]:
def create_dataset(fname):
    #fname,j,i=args
    #fname=fname+"/"+"autocon.dat"
    if os.path.isfile(fname):
        print(fname)
        infile = open(fname, 'r')
        data=[]
        for line in infile:
            #print (line)
            data.append(line) #list containing all data
            
        num_data=data[3:-1]
        num_data_list=[]
        for num in num_data: #For reading only the numerical data
            k=num.split(" ")
            ls=[]
            for c in k:
                if c!='': #Note: this is an empty string
                    ls.append(float(c))
            num_data_list.append(ls)
            
        arr_numdata=np.asarray(num_data_list)
        multipl=arr_numdata.shape[0]
        print (multipl)
        #a = np.empty(multipl, dtype=object) 
        #print(arr_numdata)
        #For writing the values of mu etc
        #params=[]
        k=data[1].split(" ")
        for p in k:
            if p!='':
                #params.append(float(p))
                a=np.full((multipl,1), float(p))
                arr_numdata=np.append(arr_numdata,a,axis=1)
                
                
    #temp[:,:,j,i]=delayed(np.asarray(arr_numdata))
    return arr_numdata

In [19]:

column_names = ["time","Lc","Nc","Zc","Lr","Nr","Zr","auto_num","auto_mass","accr_num","accr_mass","self_cloud","self_rain","nsp", "lwc0","rm", "nu_gam", "mu_gam"]


In [53]:
ncfile = Dataset("/gpfs/work/sharmas/mc-snow-data/same_sim.nc",mode='w',format='NETCDF4_CLASSIC') 
print(ncfile)

HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 140043272513344:
  #000: H5F.c line 444 in H5Fcreate(): unable to create file
    major: File accessibilty
    minor: Unable to open file
  #001: H5Fint.c line 1364 in H5F__create(): unable to open file
    major: File accessibilty
    minor: Unable to open file
  #002: H5Fint.c line 1579 in H5F_open(): unable to truncate a file which is already open
    major: File accessibilty
    minor: Unable to open file


PermissionError: [Errno 13] Permission denied: b'/gpfs/work/sharmas/mc-snow-data/same_sim.nc'

In [37]:
ensemble_dim = ncfile.createDimension('ensemble', 960)     # Ensemble axis
run_dim = ncfile.createDimension('run', 1)# Unique simulations axis
moments_dim = ncfile.createDimension('moments', 18)
time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).
for dim in ncfile.dimensions.items():
    print(dim)

('ensemble', <class 'netCDF4._netCDF4.Dimension'>: name = 'ensemble', size = 960)
('run', <class 'netCDF4._netCDF4.Dimension'>: name = 'run', size = 1)
('moments', <class 'netCDF4._netCDF4.Dimension'>: name = 'moments', size = 18)
('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 0)


In [38]:
ncfile.title='Superdroplet data with 100 sim and a bigger box and seeded'
print(ncfile.title)
ncfile.subtitle="All Simulations"
print(ncfile.subtitle)
print(ncfile)

Superdroplet data with 100 sim and a bigger box and seeded
All Simulations
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    title: Superdroplet data with 100 sim and a bigger box and seeded
    subtitle: All Simulations
    dimensions(sizes): ensemble(960), run(1), moments(18), time(0)
    variables(dimensions): 
    groups: 


In [39]:
ensemble = ncfile.createVariable('ensemble', np.float64, ('ensemble',))
ensemble.units = 'Unitless'
ensemble.long_name = 'ensemble number'


run = ncfile.createVariable('run', np.float64, ('run',))
run.units = 'Unitless'
run.long_name = 'Unique Simulations'

moments = ncfile.createVariable('moments', np.float64, ('moments',))
moments.units = 'Mixed Units'
moments.long_name = '["Lc","Nc","Zc","Lr","Nr","Zr","auto_num","auto_mass","accr_num","accr_mass","self_cloud","self_rain","nsp", "lwc0","rm", "nu_gam", "mu_gam"]'


time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'Seconds since start of the simulations'
time.long_name = 'time of simulation'
temp = ncfile.createVariable('temp',np.float64,('time','moments','run','ensemble')) # note: unlimited dimension is leftmost
temp.units = "Mixed Units"

temp.standard_name = 'Per Simulations dataset'
print(temp)

<class 'netCDF4._netCDF4.Variable'>
float64 temp(time, moments, run, ensemble)
    units: Mixed Units
    standard_name: Per Simulations dataset
unlimited dimensions: time
current shape = (0, 18, 1, 960)
filling on, default _FillValue of 9.969209968386869e+36 used


In [1]:
from dask.distributed import Client

client = Client(n_workers=48)

In [6]:
from dask import delayed

In [31]:
from dask import compute

In [16]:
global temp

In [17]:
temp

<class 'netCDF4._netCDF4.Variable'>
float64 temp(time, moments, run, ensemble)
    units: Mixed Units
    standard_name: Per Simulations dataset
unlimited dimensions: time
current shape = (0, 18, 819, 100)
filling on, default _FillValue of 9.969209968386869e+36 used

In [34]:
from multiprocessing.dummy import Pool

In [37]:

moments_data_all=[]
for i in range (0,1):
    #moments_per_run=[]
    file_num=str(i).zfill(2)
    base_dir="/gpfs/work/sharmas/experiments/data"+file_num
    dir_list=sorted(os.listdir(base_dir))
    j_list=list(np.arange(0,819))
    
    i_list=list(np.full((819,1),i))
    allList =list( zip(dir_list,j_list,i_list))
    
    pool = Pool(48)
    pool.map(create_dataset, allList,chunksize=1)
    pool.close()
    pool.join()
                        
    

UnboundLocalError: local variable 'arr_numdata' referenced before assignment

In [51]:

moments_data_all=[]
for i in range (0,1):
    #moments_per_run=[]
    file_num=str(i).zfill(2)
    base_dir="/gpfs/work/sharmas/experiments/data"+file_num
    dir_list=sorted(os.listdir(base_dir))
    j_list=list(np.arange(0,819))
    
    i_list=list(np.full((819,1),i))
    allList =list( zip(dir_list,j_list,i_list))
    
    
    result=compute([delayed(create_dataset)(file) for file in allList],scheduler="threads")
    

UnboundLocalError: local variable 'arr_numdata' referenced before assignment

In [40]:
for k in allList:
    print (k)

('boxwd_lwc0200_rm09_nu00_xi1024', 0, array([0]))
('boxwd_lwc0200_rm09_nu05_xi1024', 1, array([0]))
('boxwd_lwc0200_rm09_nu10_xi1024', 2, array([0]))
('boxwd_lwc0200_rm09_nu15_xi1024', 3, array([0]))
('boxwd_lwc0200_rm09_nu20_xi1024', 4, array([0]))
('boxwd_lwc0200_rm09_nu25_xi1024', 5, array([0]))
('boxwd_lwc0200_rm09_nu30_xi1024', 6, array([0]))
('boxwd_lwc0200_rm09_nu35_xi1024', 7, array([0]))
('boxwd_lwc0200_rm09_nu40_xi1024', 8, array([0]))
('boxwd_lwc0200_rm10_nu00_xi1024', 9, array([0]))
('boxwd_lwc0200_rm10_nu05_xi1024', 10, array([0]))
('boxwd_lwc0200_rm10_nu10_xi1024', 11, array([0]))
('boxwd_lwc0200_rm10_nu15_xi1024', 12, array([0]))
('boxwd_lwc0200_rm10_nu20_xi1024', 13, array([0]))
('boxwd_lwc0200_rm10_nu25_xi1024', 14, array([0]))
('boxwd_lwc0200_rm10_nu30_xi1024', 15, array([0]))
('boxwd_lwc0200_rm10_nu35_xi1024', 16, array([0]))
('boxwd_lwc0200_rm10_nu40_xi1024', 17, array([0]))
('boxwd_lwc0200_rm11_nu00_xi1024', 18, array([0]))
('boxwd_lwc0200_rm11_nu05_xi1024', 19, ar

In [54]:


j=0
moments_data_all=[]
for i in range (0,960):
    moments_per_run=[]
    file_num=str(i).zfill(3)
    base_dir="/gpfs/work/sharmas/sim_data_1/data"+file_num
    #dir_list=sorted(os.listdir(base_dir))
    dir_list=["boxwd_lwc2000_rm10_nu40_xi1024"]
    j=0
    for file in dir_list:
        fname=base_dir+"/"+file+"/"+"autocon.dat"
        if os.path.isfile(fname):
            moments_data=create_dataset(fname)
            temp[:,:,j,i]=np.asarray(moments_data)
            j+=1
     
            #moments_per_run.append(moments_data)
        

    #moments_data_all.append(moments_per_run)
            
        

/gpfs/work/sharmas/sim_data_1/data000/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data001/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data002/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data003/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data004/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data005/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data006/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data007/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data008/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data009/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data010/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data011/boxwd_lwc2000_rm10_nu40_xi1

/gpfs/work/sharmas/sim_data_1/data107/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data108/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data109/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data110/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data111/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data112/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data113/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data114/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data115/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data116/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data117/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data118/boxwd_lwc2000_rm10_nu40_xi1

/gpfs/work/sharmas/sim_data_1/data212/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data213/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data214/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data215/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data216/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data217/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data218/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data219/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data220/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data221/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data222/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data223/boxwd_lwc2000_rm10_nu40_xi1

/gpfs/work/sharmas/sim_data_1/data309/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data310/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data311/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data312/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data313/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data314/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data315/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data316/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data317/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data318/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data319/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data320/boxwd_lwc2000_rm10_nu40_xi1

/gpfs/work/sharmas/sim_data_1/data416/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data417/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data418/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data419/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data420/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data421/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data422/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data423/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data424/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data425/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data426/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data427/boxwd_lwc2000_rm10_nu40_xi1

/gpfs/work/sharmas/sim_data_1/data520/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data521/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data522/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data523/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data524/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data525/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data526/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data527/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data528/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data529/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data530/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data531/boxwd_lwc2000_rm10_nu40_xi1

194
/gpfs/work/sharmas/sim_data_1/data626/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data627/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data628/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data629/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data630/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data631/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data632/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data633/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data634/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data635/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data636/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data637/boxwd_lwc2000_rm10_nu40

/gpfs/work/sharmas/sim_data_1/data726/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data727/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data728/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data729/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data730/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data731/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data732/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data733/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data734/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data735/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data736/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data737/boxwd_lwc2000_rm10_nu40_xi1

/gpfs/work/sharmas/sim_data_1/data826/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data827/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data828/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data829/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data830/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data831/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data832/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data833/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data834/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data835/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data836/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data837/boxwd_lwc2000_rm10_nu40_xi1

/gpfs/work/sharmas/sim_data_1/data926/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data927/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data928/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data929/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data930/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data931/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data932/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data933/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data934/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data935/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data936/boxwd_lwc2000_rm10_nu40_xi1024/autocon.dat
194
/gpfs/work/sharmas/sim_data_1/data937/boxwd_lwc2000_rm10_nu40_xi1

In [55]:
temp.shape

(194, 18, 1, 960)

In [53]:
np.ma.compress_rows(l)[:,:]

array([[9.e-06, 1.e+00, 1.e+00],
       [9.e-06, 1.e+00, 1.e+00],
       [9.e-06, 1.e+00, 1.e+00],
       ...,
       [9.e-06, 1.e+00, 1.e+00],
       [9.e-06, 1.e+00, 1.e+00],
       [9.e-06, 1.e+00, 1.e+00]])

In [54]:
np.ma.getmaskarray(l)

array([[False, False, False],
       [False, False, False],
       [False, False, False],
       ...,
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [56]:
a = np.ma.MaskedArray(temp,np.ma.getmaskarray(temp))

In [57]:
np.savez_compressed('/gpfs/work/sharmas/mc-snow-data/same_sim.npz', data=temp, mask=a.mask)

In [44]:
with np.load('/gpfs/work/sharmas/mc-snow-data/same_sim.npz') as npz:
    arr = np.ma.MaskedArray(**npz)

In [48]:
arr.shape

(59, 18, 1, 960)

In [51]:
l=arr[:,-3:,0,0]

In [52]:
np.ma.compress_rows(l)

array([[1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.0e+00],
       [1.5e-05, 4.0e+00, 1.

In [66]:
arr.shape

(3599, 18, 13, 100)

In [24]:
temp.shape

(2399, 18, 13, 5)

In [67]:
temp[:,0,0,0]

masked_array(data=[2.000e+01, 4.000e+01, 6.000e+01, ..., 7.194e+04,
                   7.196e+04, 7.198e+04],
             mask=False,
       fill_value=1e+20)

In [27]:
temp.mask

True

In [68]:
np.ma.getmaskarray(arr[:,1,0,0])

array([False, False, False, ..., False, False, False])